In [2]:
import pandas as pd
import numpy as np
import os

# get current dir/filename
import inspect
this_file_path = os.path.abspath(inspect.getframeinfo(inspect.currentframe()).filename)
this_dir = os.path.dirname(this_file_path)

# project-own utils
from utils import *

class_path =os.path.join(this_dir, "data", "PP_step1_trn.class")
seq_path = os.path.join(this_dir, "data", "PP_step1_trn.fas")
protvec_file = os.path.join(this_dir, "data", "protVec_100d_3grams.csv")
# parse information
class_df = filter_unique_rownames(parse_fasta_to_df(class_path,"Binds"))
seq_df = filter_unique_rownames(parse_fasta_to_df(seq_path,"Sequence"))

# Convert levels to numeric
class_df['Binds'] = class_df['Binds'].map({"Bind" : 1, "Non_Bind" : 0})

/Users/universal/anaconda3/lib/python3.6/site-packages/Bio/Seq.py:163: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [3]:
# Convert sequences to their ProtVec representation
pv_df = import_protvec(protvec_file)
seq_df['Sequence'] = seq_df['Sequence'].apply(convert_seq_to_protvec, args = (pv_df,))

In [4]:
counts = class_df['Binds'].value_counts()
ratio = counts[0]/counts[1]
print("Bind/non-bind ratio is", str(ratio))

Bind/non-bind ratio is 0.688432835821


In [5]:
# Merge two dataframes into one 
protvec_and_target_df = pd.merge(class_df, seq_df, left_index=True, right_index=True)
protvec_and_target_df[1:10]

,Binds,Sequence
ProtID,,
P41391,1,"[-27.965312, -7.714547, 2.730592, -25.785597, ..."
Q5SJH5,1,"[-16.271814, -0.640101, 5.404184, -8.44587, -2..."
P31109,1,"[-7.48862, -3.73666, 0.497572, -9.259213, -0.1..."
P00698,1,"[-8.233843, -2.502467, -4.157566, -10.052415, ..."
Q02223,1,"[-6.723216, -2.230543, -5.659682, -13.655185, ..."
P14739,1,"[-5.44515, 0.415351, 2.689022, -6.631331, 0.54..."
Q7M418,1,"[-10.56277, -3.956826, 0.227718, -9.661443, -4..."
Q6CTI2,1,"[-14.193158, -4.443981, 0.099315, -10.816634, ..."
Q7XZK2,1,"[-9.495451, -2.119338, 3.963915, -6.633833, -3..."


In [22]:
# Eject features and targets, make CV splits
from sklearn.model_selection import StratifiedKFold
N_SPLITS_CV = 5

features = np.stack(protvec_and_target_df['Sequence'].values.tolist(), axis = 0)
targets = protvec_and_target_df['Binds'].values

skf = StratifiedKFold(n_splits=N_SPLITS_CV)
folds = skf.split(features, targets)

for train_index, test_index in folds:
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = targets[train_index], targets[test_index]

TRAIN: [181 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253
 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271
 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289
 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307
 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325
 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343
 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361
 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379
 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397
 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415
 416 417 418 419 420 421 422 423 424 425 426

In [23]:
# test layer

y_train

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [24]:
# Fit ANN # http://scikit-learn.org/stable/modules/neural_networks_supervised.html
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

X = X_train
y = y_train

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X, y)       

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [25]:
# Predict sample
clf.predict(X_test)

scores = cross_val_score(clf, features, targets, cv = skf )

In [26]:
scores

array([ 0.59340659,  0.59116022,  0.59116022,  0.59116022,  0.59444444])